<a href="https://colab.research.google.com/github/antoniosfn/-4-operations-calculator/blob/main/Normalizar_competencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab: Normalização + Adicionar ID + Mesclar múltiplos CSVs únicos em MAIÚSCULAS

In [ ]:
!pip install -q openpyxl

import io, csv
import pandas as pd
from google.colab import files

# -------------------------
# Utilitárias
# -------------------------
def drop_all_empty_columns(df):
    """
    Remove colunas cujo conteúdo é vazio/NaN em todas as linhas.
    Considera vazio se NaN ou string vazia após strip.
    """
    cols_to_keep = []
    for col in df.columns:
        ser = df[col]
        all_empty = True
        for v in ser:
            if pd.isna(v):
                continue
            if str(v).strip() == '':
                continue
            all_empty = False
            break
        if not all_empty:
            cols_to_keep.append(col)
    return df.loc[:, cols_to_keep].copy()

def pick_name_column(df):
    """
    Detecta a melhor coluna para 'nome'. Prioriza:
    unique_names_per_row, name, nome  (case-insensitive), depois primeira textual, depois primeira.
    """
    lower_map = {c.lower(): c for c in df.columns}
    for prefer in ('unique_names_per_row','name','nome'):
        if prefer in lower_map:
            return lower_map[prefer]
    for c in df.columns:
        if pd.api.types.is_string_dtype(df[c]) or pd.api.types.is_object_dtype(df[c]):
            return c
    return df.columns[0]

def split_and_unique_str(cell, sep=';'):
    """Separa por sep, strip, remove vazios e duplicatas, retorna '; ' join."""
    if pd.isna(cell):
        return ''
    s = str(cell)
    parts = [p.strip() for p in s.split(sep)]
    parts = [p for p in parts if p != '']
    seen = set()
    unique = []
    for p in parts:
        if p not in seen:
            seen.add(p)
            unique.append(p)
    return '; '.join(unique)

def robust_read_csv_from_bytes(b, enc_candidates=('utf-8-sig','utf-8','latin1')):
    """
    Lê bytes CSV com tentativas múltiplas de encoding e delimitador.
    Retorna DataFrame ou lança ValueError.
    """
    last_err = None
    for enc in enc_candidates:
        try:
            text = b.decode(enc)
        except Exception as e:
            last_err = e
            continue

        sample = text[:8192]
        # tentar sniff
        try:
            dialect = csv.Sniffer().sniff(sample, delimiters=[',',';','\t','|'])
            delim = dialect.delimiter
            try:
                df = pd.read_csv(io.StringIO(text), sep=delim, engine='c')
                return df
            except Exception:
                try:
                    df = pd.read_csv(io.StringIO(text), sep=delim, engine='python')
                    return df
                except Exception as e2:
                    last_err = e2
        except Exception as sniff_err:
            last_err = sniff_err

        # tentativas explícitas
        for sep_try in [';', ',', '\t', '|']:
            try:
                df = pd.read_csv(io.StringIO(text), sep=sep_try, engine='c')
                return df
            except Exception as e:
                last_err = e
                try:
                    df = pd.read_csv(io.StringIO(text), sep=sep_try, engine='python')
                    return df
                except Exception as e2:
                    last_err = e2

        # fallback final para esse encoding
        try:
            df = pd.read_csv(io.StringIO(text), sep=None, engine='python')
            return df
        except Exception as e:
            last_err = e

    raise ValueError("Não foi possível interpretar o CSV com as estratégias tentadas. Último erro: " + repr(last_err))

# -------------------------
# Fluxo principal
# -------------------------
print("Escolha uma opção:")
print("1 - Ler .xlsx, normalizar (gera normalized_name_only.csv com coluna 'nome')")
print("2 - Ler normalized_name_only.csv, adicionar ID (gera csv_with_id_and_name.csv com colunas 'id','nome')")
print("3 - Ler múltiplos .csv, mesclar únicos em MAIÚSCULAS (gera unique_merged_uppercase.csv com coluna 'nome')")

_mode = input("Digite 1, 2 ou 3 e pressione Enter: ").strip()
if _mode not in ('1','2','3'):
    print("Entrada inválida. Usando opção 1 por padrão.")
    _mode = '1'

# -------------------------
# Opção 1: XLSX -> normalized_name_only.csv (coluna 'nome')
# -------------------------
if _mode == '1':
    print("\n== Opção 1: upload do .xlsx ==")
    print("Envie o arquivo .xlsx agora.")
    uploaded = files.upload()
    if not uploaded:
        raise SystemExit("Nenhum arquivo enviado. Abortando.")

    # encontrar primeiro arquivo Excel
    xlsx_file = None
    for f in uploaded.keys():
        if f.lower().endswith(('.xlsx','.xlsm','.xls')):
            xlsx_file = f
            break
    if xlsx_file is None:
        raise SystemExit("Nenhum arquivo Excel (.xlsx/.xlsm/.xls) encontrado no upload.")

    df = pd.read_excel(io.BytesIO(uploaded[xlsx_file]), engine='openpyxl')
    print(f"[OK] Arquivo carregado: {xlsx_file} — shape: {df.shape}")

    # remover colunas vazias
    df = drop_all_empty_columns(df)
    print(f"[OK] Após remover colunas vazias — shape: {df.shape}")

    # detectar coluna de nomes
    chosen_col = pick_name_column(df)
    print(f"[OK] Coluna escolhida: '{chosen_col}'")

    # normalizar para coluna 'nome'
    df['nome'] = df[chosen_col].apply(lambda x: split_and_unique_str(x, sep=';'))

    # manter apenas 1 coluna: nome
    df_out = df[['nome']].copy()

    out_name = "normalized_name_only.csv"
    df_out.to_csv(out_name, index=False, encoding='utf-8-sig')
    print(f"[OK] Exportado: {out_name} (contém coluna 'nome')")
    files.download(out_name)
    display(df_out.head(20))

# -------------------------
# Opção 2: normalized_name_only.csv -> csv_with_id_and_name.csv (colunas 'id','nome')
# -------------------------
if _mode == '2':
    print("\n== Opção 2: upload do .csv já normalizado ==")
    print("Envie o arquivo normalized_name_only.csv (ou equivalente).")
    uploaded = files.upload()
    if not uploaded:
        raise SystemExit("Nenhum arquivo enviado. Abortando.")

    # escolher primeiro CSV
    csv_file = None
    for f in uploaded.keys():
        if f.lower().endswith('.csv'):
            csv_file = f
            break
    if csv_file is None:
        csv_file = list(uploaded.keys())[0]
    print(f"[INFO] Lendo: {csv_file}")

    try:
        df = robust_read_csv_from_bytes(uploaded[csv_file])
    except Exception as e:
        raise SystemExit(f"Falha ao ler CSV: {e}")

    print(f"[OK] CSV lido — shape: {df.shape}")

    # remover colunas vazias
    df = drop_all_empty_columns(df)
    print(f"[OK] Após remover colunas vazias — shape: {df.shape}")

    # detectar coluna de nomes
    name_col = pick_name_column(df)
    print(f"[OK] Usando coluna '{name_col}' como fonte de nomes.")

    # normalizar a coluna nome (garantir formato consistente)
    df['nome'] = df[name_col].apply(lambda x: split_and_unique_str(x, sep=';'))

    # pedir start_id
    while True:
        sval = input("Digite o ID inicial (ex.: 124) e pressione Enter: ").strip()
        try:
            start_id = int(sval)
            break
        except:
            print("Valor inválido. Digite um número inteiro.")

    # perguntar se atribuir IDs apenas às linhas não-vazias
    apply_only_nonempty = input("Atribuir IDs apenas às linhas com 'nome' não vazia? (s/N): ").strip().lower() or 'n'

    df = df.reset_index(drop=True)
    if apply_only_nonempty.startswith('s'):
        ids = [''] * len(df)
        current = start_id
        for i, val in enumerate(df['nome'].astype(str)):
            if pd.isna(val) or str(val).strip() == '':
                ids[i] = ''
            else:
                ids[i] = current
                current += 1
        df['id'] = ids
        print(f"[OK] IDs atribuídos apenas a linhas não-vazias, começando em {start_id}.")
    else:
        df['id'] = [start_id + i for i in range(len(df))]
        print(f"[OK] IDs atribuídos a todas as linhas, começando em {start_id}.")

    # saída com no máximo 2 colunas: id, nome
    df_out = df[['id', 'nome']].copy()
    out_name = "csv_with_id_and_name.csv"
    df_out.to_csv(out_name, index=False, encoding='utf-8-sig')
    print(f"[OK] Exportado: {out_name} (colunas: 'id','nome')")
    files.download(out_name)
    display(df_out.head(20))

# -------------------------
# Opção 3: múltiplos CSVs -> unique_merged_uppercase.csv (coluna 'nome' em MAIÚSCULAS)
# -------------------------
if _mode == '3':
    print("\n== Opção 3: upload de múltiplos .csv ==")
    print("Selecione e envie todos os arquivos .csv que deseja mesclar (pode selecionar vários).")
    uploaded = files.upload()
    if not uploaded:
        raise SystemExit("Nenhum arquivo enviado. Abortando.")

    # coletar CSVs enviados
    csv_keys = [k for k in uploaded.keys() if k.lower().endswith('.csv')]
    if not csv_keys:
        # se nenhum com .csv, tentar usar todos
        csv_keys = list(uploaded.keys())

    all_names = []
    seen = set()
    for k in csv_keys:
        print(f"[INFO] Lendo {k} ...")
        try:
            df_tmp = robust_read_csv_from_bytes(uploaded[k])
        except Exception as e:
            print(f"[AVISO] Falha ao ler {k}: {e}. Pulando.")
            continue

        # remover colunas vazias
        df_tmp = drop_all_empty_columns(df_tmp)

        if df_tmp.shape[1] == 0:
            print(f"[AVISO] {k} não tem colunas não-vazias. Pulando.")
            continue

        # detectar coluna de nomes
        name_col = pick_name_column(df_tmp)
        # extrair valores, separar por ';' (se houver), strip e uppercase
        for cell in df_tmp[name_col].dropna().astype(str):
            parts = [p.strip() for p in cell.split(';') if p.strip() != '']
            for p in parts:
                up = p.upper()
                if up not in seen:
                    seen.add(up)
                    all_names.append(up)

    if not all_names:
        raise SystemExit("Nenhum nome único encontrado nos arquivos enviados.")

    df_out = pd.DataFrame({'nome': all_names})
    out_name = "unique_merged_uppercase.csv"
    df_out.to_csv(out_name, index=False, encoding='utf-8-sig')
    print(f"[OK] Exportado: {out_name} (total únicos: {len(all_names)}) — coluna 'nome' em MAIÚSCULAS")
    files.download(out_name)
    display(df_out.head(50))

print("\nConcluído ✅")


Escolha uma opção:
1 - Ler .xlsx, normalizar (gera normalized_name_only.csv com coluna 'nome')
2 - Ler normalized_name_only.csv, adicionar ID (gera csv_with_id_and_name.csv com colunas 'id','nome')
3 - Ler múltiplos .csv, mesclar únicos em MAIÚSCULAS (gera unique_merged_uppercase.csv com coluna 'nome')
